In [7]:
import os
import requests
import os.path
import csv
import json
import pytz
import pygsheets
from nbdev import *

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta
import pandas as pd

##### Copper Code and Functions #####

headers = {
    'X-PW-AccessToken':os.environ["COPPER_API_KEY"],
    'X-PW-Application':'developer_api',
    'X-PW-UserEmail':'cooper@rehabpath.com',
    'Content-Type':'application/json'
}

def refresh_custom_fields(return_results:bool = False, #Whether you would like results returned
):
    """
    Function to refresh the custom_field_json file and related dictionary. Option to 
    return resulting dictionary.
    """
    # Fetching custom fields:
    list_fields = requests.get('https://api.copper.com/developer_api/v1/custom_field_definitions',headers=headers)
    
    # Was the API request a success?
    if list_fields.status_code != 200:
        print('Error fetching custom fields.')
        if return_results:
            print('Returning previous version.')
            with open('custom_fields.json', encoding="utf-8") as f:
                return json.load(f)
    
    # Obtaining response text and initializing dictionary:
    list_fields_json = list_fields.json()
    custom_field_dict = {}

    for item in list_fields_json:
        item_dict = {}
        for sub_item in ['name','data_type','available_on','is_filterable']:
            item_dict[sub_item] = item[sub_item] 

        item_options = {}

        if 'options' in item:
            for sub_item in item['options']:
                sub_item_name = sub_item['name']
                sub_item_id = sub_item['id']

                item_options[sub_item_id] = sub_item_name
        
            item_dict['options'] = item_options
        item_id = item['id']
        custom_field_dict[item_id] = item_dict
    
    # Updating local file:
    with open('custom_fields.json', 'w') as file:
        json.dump(custom_field_dict, file, indent=4)
    
    #Option to return results or just update local file:
    if return_results == True:
        return custom_field_dict
    else:
        print('Successfully Updated File!')
        pass

custom_fields = refresh_custom_fields(True)

def get_cf_info(cf_id:str,     #ID of custom field
                cf_info:list,  #Designed information about field, list if multiple items
)->list: #Returns list if cf_info is list. Otherwise, returns value
    """
    Function to get the custom field information based on the field id. Can be supplied with a sigular value for 'cf_info' or multiple as a list.
    """

    if isinstance(cf_id,str):
        cf_id = int(cf_id)      

    # Check if cf_info item(s) are valid
    if isinstance(cf_info, list):
        for item_name in cf_info:
            # Remove faulty items:
            if item_name not in ['name','data_type','available_on','is_filterable','options']:
                cf_info.pop(item_name)
                print(f'Invalid cf_info: {cf_info}')
            
            # Cancel function if no valid items
            if len(cf_info) == 0:
                pass
    elif cf_info not in ['name','data_type','available_on','is_filterable','options']:
        print(f'Invalid cf_info: {cf_info}'); pass

    if isinstance(cf_info,list):
        return_list = []
        for item_name in cf_info:
            value = custom_fields[cf_id].get(item_name)
            return_list.append(value)
        return return_list
    else:
        value = custom_fields[cf_id].get(cf_info)
        return value

def get_cf_options(cf_id:str, #Coppper ID for custom field
)->dict: # Returns dictionary of options for field
    return custom_fields.get(cf_id).get('options')

def cf_option_name(cf_id:str, #Coppper ID for custom field
option_id:str, #Coppper ID for option
)->str: #Returns name/value of option
    return get_cf_options(cf_id).get(option_id)

def reformat_company_data(company_data:list, #Returned company data as list of dictionaries.
):
    """
    Function to reformat the company data recieved from Copper in its JSON form 
    and clean the results.
    """

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items:
        output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)

        if item_name not in Custom_Fields or item_name is None:
            continue
        
        if item_name is not None:
            item_value = dict_item['value']
            output_dict[item_name] = item_value
    
    return output_dict

def write_to_gsheet(service_file_path:str,#Path to credentials json file
 spreadsheet_id:str, #ID of spreadhseet. Found in url to gsheet after '/d/'
  sheet_name:str, #Name location where you want the data sent.
  data_df:pd.DataFrame, #Data to be send.
  ):
    """
    Function takes data_df and writes it to the provided spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

##### Minor code to update custom fields stuff ####

custom_fields_dict = {}

for item in custom_fields.keys():
    item_name = custom_fields[item].get('name')
    custom_fields_dict[item] = item_name

custom_fields_df = pd.DataFrame(list(custom_fields_dict.items()),columns = ['id','name'])

custom_fields_list = list(custom_fields_df['name'])

##### Start of Google Sheet Functions #####

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

def authorize_gspread(credentials_file, token_file, scopes):
    creds = None

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, scopes)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, scopes)
            creds = flow.run_local_server(port=0)

        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return creds


def read_google_sheets(spreadsheet_id, range_name, credentials_file, token_file, scopes):
    creds = authorize_gspread(credentials_file, token_file, scopes)

    try:
        service = build("sheets", "v4", credentials=creds)

        sheet = service.spreadsheets()
        resulting_sheet = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
        values = resulting_sheet.get("values", [])

        if not values:
            return None

        return values

    except HttpError as err:
        return None


def get_data(spreadsheet_id:str,#ID of spreadhseet. Found in url to gsheet after '/d/'
range_name:str,#Name location where you want the data sent.
credentials_file:str= "credentials.json", #path to json credentials file.
token_file:str = "token.json", #path to token json file
)->pd.DataFrame: #Returns results as DataFrame
    """
    Function to fetch data from Google Sheets.
    """
    result = read_google_sheets(spreadsheet_id, range_name, credentials_file, token_file, SCOPES)
    
    if result is not None:
        results = pd.DataFrame(result[1:], columns=result[0])
        return results
    else:
        print("Failed to retrieve data from Google Sheets.")
        pass


# Daily Job to Get PPV Values

Daily job to pull GA4 pageviews from snowflake and combine it with data from Copper to get the total PPVs by day for all advertier and send the results to `AdvertisingPartners_RatesAndBudgets`.

**Data from Snowflake:**
- Pageviews by `slug` by day

**Data from Coppper:**
- Center `slug` - to pair to GA4 data
- Center `GoLive Date` - to discard PPVs before a advertier goes GoLive
- Center `Page` - similar to slug, but how the data was been historically reported.

### Getting Data from Copper

Let's start by getting the Copper data for all companies with the status of `Advertiser`, `Onboarding-CS`,`Onboarding-AdOps`, `Paused`, or `Churning`.  We can do this by searching for comapnies with these values in the field. Note, that we need to use the id for the custome field, 588393, and the ids for all of the statuses.

In [8]:
total_pages = page = 1
combined_results = []

while page <= total_pages:
    page_params = {
        "page_size": 100,
        "page_number": page,
        "custom_fields": [
            {"custom_field_definition_id": 588393,                  # Company Status - custom field
            "value": [1806073, 1806063,1806064,1824709,1806067]    # List of ids for allowed statuses
        }
        ]}
    result = requests.post('https://api.copper.com/developer_api/v1/companies/search',headers=headers,json=page_params)
    
    if result.status_code == 200:
        total_pages = (int(result.headers['X-PW-TOTAL'])//100)+1
        print(f"Getting page {page} of {total_pages}.")
        result_json = result.json()
        combined_results.extend(result_json)
        page +=1

    else:
        print(f"Issue with page {page}. Stopping.")
        break

Getting page 1 of 3.
Getting page 2 of 3.
Getting page 3 of 3.


Now that we have the data we want, we need to do some data cleaning to dates and custom fields and discard information we don't want. First, let's loop through the results, which are in a json/dict format, turn them into a flat list using `reformat_company_data`, and finally turn the list into a dataframe.

In [9]:
Custom_Fields = ['GoLive Date','Page','Profile Slug']

In [10]:
show_doc(reformat_company_data)

---

### reformat_company_data

>      reformat_company_data (company_data:list)

Function to reformat the company data recieved from Copper in its JSON form 
and clean the results.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| company_data | list | Returned company data as list of dictionaries. |

In [ ]:
cleaned_data = []

for idx in range(len(combined_results)):
    results = reformat_company_data(combined_results[idx]) # using a function I wrote. See hidden section.
    cleaned_data.append(results)

cleaned_data_df = pd.DataFrame(cleaned_data)
list_slugs = list(set(cleaned_data_df['Profile Slug']))
cleaned_data_df.drop(columns=['id','name','address','assignee_id','contact_type_id'],inplace=True)

Now that we have a dataframe, let's change the data formate from a unix timestamp to a human readable date.

In [ ]:
ct_timezone = pytz.timezone('US/Central')
cleaned_data_df['GoLive Date'] = cleaned_data_df['GoLive Date'].apply(lambda x: pd.to_datetime(x, unit='s')).dt.strftime("%Y-%m-%d")
cleaned_data_df

,Profile Slug,GoLive Date,Page
0,1000-islands,1999-12-28,/1000-islands/
1,90210-recovery-california,1999-12-31,/90210-recovery-california/
2,abhasa-rehabilitation-thondamuthur-india,2023-03-25,/abhasa-rehabilitation-thondamuthur-india/
3,None,2023-02-28,None
4,abhasa-wellness-retreat,2022-10-16,/abhasa-wellness-retreat/
...,...,...,...
293,wish-recovery-california,2023-07-14,/wish-recovery-california/
294,within-center-austin-texas,2023-10-08,/within-center-austin-texas/
295,zeus-rehab-warsaw-poland,2024-01-04,/zeus-rehab-warsaw-poland/
296,zinnia-healing-singer-island,2023-05-05,/zinnia-healing-singer-island/


### Getting Views from Snowflake

We can pull all of the pageviews / ppvs data from snowflake by inserting dynamic values into the SQL query. We want all of the data for yesterday's month and year. This get's the whole month of data even when yesterday was the last day of the month.

In [ ]:
yesterday = datetime.now(ct_timezone)-timedelta(1)
search_year = yesterday.year
search_month = yesterday.month

We can insert dynamic values into the SQL querry by using double curly bracekts {{example}}.

In [ ]:
views_df = _deepnote_execute_sql('SELECT * \nFROM RP_PROD_DL."CLEAN_TABLES_GA4".CENTER_DAILY_VIEWS \nWHERE YEAR(AS_OF_DATE) = {{search_year}} AND MONTH(AS_OF_DATE) = {{search_month}};', 'SQL_5E90C4C2_A521_4993_A1E9_D77837A4FD79', audit_sql_comment='', sql_cache_mode='cache_disabled')
views_df

,as_of_date,slug,property_id,region,city,number_of_views
0,2024-01-04,browse,278764105,Missouri,Boonville,4
1,2024-01-04,greater-boston-behavioral-health-needham-massa...,278764105,Massachusetts,Boston,3
2,2024-01-04,,278764105,Tennessee,Knoxville,5
3,2024-01-04,luxe-recovery-california,278764105,California,San Diego,2
4,2024-01-04,,278764105,Centre-Val de Loire,Pithiviers,1
...,...,...,...,...,...,...
38074,2024-01-07,my-time-recovery-california,278764105,California,Fresno,1
38075,2024-01-07,south-florida,278764105,Georgia,St. Simons Island,2
38076,2024-01-07,spain,278764105,County Dublin,Dublin,1
38077,2024-01-07,the-orchard-on-the-brazos,278764105,Texas,Wichita Falls,1


Now that we have the data, let's again do some data cleaning. We need to turn `as_of_date` into a date datatype, discard data for non-advertisers and listing pages, and transform the data into a table.

In [ ]:
# Formating Date:
views_df['as_of_date'] = views_df['as_of_date'].astype(str)

# Discarding unneeded data:
filtered_views = views_df[views_df['slug'].isin(list_slugs)][['as_of_date', 'slug', 'number_of_views']]

# Formating into a table:
pivoted_views = filtered_views.groupby(['slug', 'as_of_date'])['number_of_views'].sum().unstack().reset_index()
pivoted_views.rename(columns={'slug':'Profile Slug'},inplace=True)
pivoted_views['Page'] = "/"+pivoted_views['Profile Slug']+"/"
pivoted_views

as_of_date,Profile Slug,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,Page
0,1000-islands,28.0,18.0,18.0,12.0,10.0,13.0,27.0,6.0,/1000-islands/
1,90210-recovery-california,5.0,2.0,4.0,4.0,2.0,6.0,7.0,NaN,/90210-recovery-california/
2,abhasa-rehabilitation-india,15.0,11.0,7.0,8.0,7.0,12.0,16.0,1.0,/abhasa-rehabilitation-india/
3,abhasa-rehabilitation-thondamuthur-india,8.0,6.0,3.0,6.0,1.0,12.0,11.0,NaN,/abhasa-rehabilitation-thondamuthur-india/
4,abhasa-wellness-retreat,7.0,5.0,7.0,2.0,3.0,13.0,12.0,1.0,/abhasa-wellness-retreat/
...,...,...,...,...,...,...,...,...,...,...
250,wilmington-treatment-center-north-carolina,1.0,3.0,NaN,2.0,2.0,3.0,1.0,NaN,/wilmington-treatment-center-north-carolina/
251,wish-recovery-california,21.0,13.0,16.0,17.0,17.0,33.0,30.0,NaN,/wish-recovery-california/
252,within-center-austin-texas,4.0,3.0,11.0,13.0,8.0,8.0,19.0,1.0,/within-center-austin-texas/
253,zeus-rehab-warsaw-poland,5.0,1.0,2.0,3.0,7.0,13.0,12.0,NaN,/zeus-rehab-warsaw-poland/


### Removing PPVs Before GoLive Dates

Our PPV data is now ready to be combined with the GoLive dates. This will allow us to search whether a column is before a center's GoLive date. If it is, we're going to set the PPVs on that day to zero.

In [ ]:
# Mergeing/joining our two dataframes:
merged_df = pd.merge(pivoted_views, cleaned_data_df, on='Page', how='left')
list_cols = list(merged_df.columns[1:])
list_cols.remove('GoLive Date')
list_cols.remove('Page')
list_cols.remove('Profile Slug_y')

# Discarding extra columns and moving 'Page' to front:
merged_df = merged_df[['Page','GoLive Date'] + list(list_cols)]

All dates currently in Snowflake for month:

In [ ]:
list_cols

['2024-01-01',
 '2024-01-02',
 '2024-01-03',
 '2024-01-04',
 '2024-01-05',
 '2024-01-06',
 '2024-01-07',
 '2024-01-08']

Since GoLive Date has been moved, we need to turn it into a date, again, but it is already formated correctly from our ealier work. 

We are then ready to use the column dates and `Go Live Dates` to discard PPVs before a center was live.

Getting Exmpetion dates from Oscar Diggs by using `get_data` function:

In [13]:
show_doc(get_data)

---

### get_data

>      get_data (spreadsheet_id:str, range_name:str,
>                credentials_file:str='credentials.json',
>                token_file:str='token.json')

Function to fetch data from Google Sheets.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| spreadsheet_id | str |  | ID of spreadhseet. Found in url to gsheet after '/d/' |
| range_name | str |  | Name location where you want the data sent. |
| credentials_file | str | credentials.json | path to json credentials file. |
| token_file | str | token.json | path to token json file |
| **Returns** | **DataFrame** |  | **Returns results as DataFrame** |

In [ ]:
# Getting Data:
exemption_data = get_data('17BIW8GWs5Ap9XGZL4ms8kg7ZNfYA8yRZtQtnCKgXWi8','Testing_Exemptions (DaveK Example)')

Now that we have the data, we need to do some data cleaning:
- Filter out unrelated exemptions
- Turn `Exemption Start` and `Exemption End` into dates
- Fitler exemptions to only those in the relevant month

In [ ]:
# Check if 'Exemption' column is present in dataframe before proceeding
exemption_data_filtered = exemption_data[exemption_data['Exemption'] == 'Exclude from Billable'].copy()
exemption_data_filtered.drop(columns=['Exemption','Exemption Value'], inplace=True)

# Setting Start and End columns to date datattype:
exemption_data_filtered['Exemption Start'] = pd.to_datetime(exemption_data_filtered['Exemption Start'])
exemption_data_filtered['Exemption End'] = pd.to_datetime(exemption_data_filtered['Exemption End'])

# Filtering to only relevant exemptions:
exemption_data_filtered = exemption_data_filtered[
    (exemption_data_filtered['Exemption Start'].dt.year == search_year) | 
    (exemption_data_filtered['Exemption End'].dt.year == search_year)
]
exemption_data_filtered = exemption_data_filtered[
    (exemption_data_filtered['Exemption Start'].dt.month == search_month) | 
    (exemption_data_filtered['Exemption End'].dt.month == search_month)
]

We also need to turn `Slug` into `Page` so that it is alaigned with merged_df which uses Page.

In [ ]:
exemption_data_filtered['Page'] = "/"+ exemption_data_filtered['Slug']+"/"
exemption_data_filtered.drop(columns='Slug',inplace=True)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

list_of_changes = [] 
def check_dates(row, list_cols, exemption_data):
    go_live_date = pd.to_datetime(row['GoLive Date']) + timedelta(days=1)
    center_page = row['Page']
    if pd.isnull(go_live_date):
        for col in list_cols:
            row[col] = ''
            list_of_changes.append([center_page,col,"Not Live - Note GoLive Date"])
    else:
        for col in list_cols:
            col_date = datetime.strptime(col, "%Y-%m-%d")
            if col_date < go_live_date:
                row[col] = ''
                list_of_changes.append([center_page,col,"Before Golive"])
        
            if len(exemption_data_filtered) == 0:
                break

            center_exemptions = exemption_data_filtered[exemption_data_filtered['Page'] == row['Page']].copy()
            if len(filtered_exemptions) == 0:
                break

            for _, sub_row in center_exemptions.iterrows():
                exemption_start = pd.to_datetime(sub_row['Exemption Start'])
                exemption_end = pd.to_datetime(sub_row['Exemption End'])
                if col_date >= exemption_start and col_date <= exemption_end:
                    row[col] = ''
                    list_of_changes.append([center_page,col,"Date Excluded"])
                    break  # Exit the loop once a match is found

    return row

def safe_date_check(row, list_cols):
    go_live_date = pd.to_datetime(row['GoLive Date']) + timedelta(days=1)
    center_page = row['Page']
    if pd.isnull(go_live_date):
        for col in list_cols:
            row[col] = ''
            list_of_changes.append([center_page,col,"Not Live - Note GoLive Date"])
    else:
        for col in list_cols:
            col_date = datetime.strptime(col, "%Y-%m-%d")
            if col_date < go_live_date:
                row[col] = ''
                list_of_changes.append([center_page,col,"Before Golive"])
        
# Apply the function
try:
    merged_df = merged_df.apply(lambda row: check_dates(row, list_cols, exemption_data_filtered), axis=1)
except Exception as e:
    print(f'Error encountered.\n {e} ')
    print('Revering to just excluding dates. Manual edits needed.')
    merged_df = merged_df.apply(lambda row: safe_date_check(row, list_cols), axis=1)

df_changes = pd.DataFrame(list_of_changes)
df_changes.rename(columns={0:"Page",1:'Date',2:'Reason'})

,Page,Date,Reason
0,/agape-treatment-center-fort-lauderdale-florida/,2024-01-01,Not Live - Note GoLive Date
1,/agape-treatment-center-fort-lauderdale-florida/,2024-01-02,Not Live - Note GoLive Date
2,/agape-treatment-center-fort-lauderdale-florida/,2024-01-03,Not Live - Note GoLive Date
3,/agape-treatment-center-fort-lauderdale-florida/,2024-01-04,Not Live - Note GoLive Date
4,/agape-treatment-center-fort-lauderdale-florida/,2024-01-05,Not Live - Note GoLive Date
...,...,...,...
186,/white-deer-run-lancaster-pennsylvania/,2024-01-05,Not Live - Note GoLive Date
187,/white-deer-run-lancaster-pennsylvania/,2024-01-06,Not Live - Note GoLive Date
188,/white-deer-run-lancaster-pennsylvania/,2024-01-07,Not Live - Note GoLive Date
189,/white-deer-run-lancaster-pennsylvania/,2024-01-08,Not Live - Note GoLive Date


Because of the quirks of Google Sheets, I'm going to add empty columns to the date of the dataframe to ensure the total number of columns is always equal to 33. This ensures that Dave's formulas for finding the PPVs still work.

In [ ]:
def ensure_32_columns(df):
    current_column_count = df.shape[1]
    columns_to_add = 33 - current_column_count

    for i in range(columns_to_add):
        df[f'Empty Column {current_column_count + i + 1}'] = pd.NA

    return df


merged_df = ensure_32_columns(merged_df)
merged_df = merged_df.where(pd.notnull(merged_df), '')

After dropping `Go Live Date`, or data is finalized and ready to be sent to the APRB sheet:

In [ ]:
finished_data = merged_df.drop(columns=['GoLive Date'])
finished_data

,Page,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,Empty Column 11,...,Empty Column 24,Empty Column 25,Empty Column 26,Empty Column 27,Empty Column 28,Empty Column 29,Empty Column 30,Empty Column 31,Empty Column 32,Empty Column 33
0,/1000-islands/,28.0,18.0,18.0,12.0,10.0,13.0,27.0,6.0,,...,,,,,,,,,,
1,/90210-recovery-california/,5.0,2.0,4.0,4.0,2.0,6.0,7.0,,,...,,,,,,,,,,
2,/abhasa-rehabilitation-india/,15.0,11.0,7.0,8.0,7.0,12.0,16.0,1.0,,...,,,,,,,,,,
3,/abhasa-rehabilitation-thondamuthur-india/,8.0,6.0,3.0,6.0,1.0,12.0,11.0,,,...,,,,,,,,,,
4,/abhasa-wellness-retreat/,7.0,5.0,7.0,2.0,3.0,13.0,12.0,1.0,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,/wilmington-treatment-center-north-carolina/,1.0,3.0,,2.0,2.0,3.0,1.0,,,...,,,,,,,,,,
251,/wish-recovery-california/,21.0,13.0,16.0,17.0,17.0,33.0,30.0,,,...,,,,,,,,,,
252,/within-center-austin-texas/,4.0,3.0,11.0,13.0,8.0,8.0,19.0,1.0,,...,,,,,,,,,,
253,/zeus-rehab-warsaw-poland/,,1.0,2.0,3.0,7.0,13.0,12.0,,,...,,,,,,,,,,


### Send Data to ADPB Sheet

Sending data to AdvertisingPartners_RatesAndBudgets using the GoogleSheets API and pygsheets:

In [12]:
show_doc(write_to_gsheet)

---

### write_to_gsheet

>      write_to_gsheet (service_file_path:str, spreadsheet_id:str,
>                       sheet_name:str, data_df:pandas.core.frame.DataFrame)

Function takes data_df and writes it to the provided spreadsheet_id
and sheet_name using your credentials under service_file_path

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| service_file_path | str | Path to credentials json file |
| spreadsheet_id | str | ID of spreadhseet. Found in url to gsheet after '/d/' |
| sheet_name | str | Name location where you want the data sent. |
| data_df | DataFrame | Data to be send. |

In [ ]:
write_to_gsheet('/work/keveldata-5f76a71e9aaf.json','1G5Mo3wrT_SrbFt5lyjKDne4C-Hj0cfjxCaW9mGPLQXU',"PPV Data",finished_data)

In [ ]:
today = datetime.now(ct_timezone)

directory = "/datasets/coopers-google-drive/Projects/revops_data/ppv_history/"

if not os.path.exists(directory):
    os.makedirs(directory)
    
df_changes.to_csv(f"{directory}/{today.strftime('%Y_%m_%d')}_history.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d4ba941-f702-478c-87bf-a3e3e8035885' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>